In [2]:
# Python library imports: numpy, random, sklearn, pandas, etc

import warnings
warnings.filterwarnings('ignore')

from datetime import timedelta
from datetime import datetime
import dateutil.parser
import sys
import random
import numpy as np

from sklearn import linear_model, cross_validation, metrics, svm
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from lxml import objectify

path = '/home/sohail/speedmap.xml'
xml = objectify.parse(open(path))

root = xml.getroot()
root.getchildren()[0]

dfSpeedMap = pd.DataFrame(columns=('LINK_ID', 'REGION','ROAD_TYPE', 'ROAD_SATURATION_LEVEL', 'TRAFFIC_SPEED', 'CAPTURE_DATE'))

for i in range(0,617):
    obj = root.getchildren()[i].getchildren()
    row = dict(zip(['LINK_ID', 'REGION','ROAD_TYPE', 'ROAD_SATURATION_LEVEL', 'TRAFFIC_SPEED', 'CAPTURE_DATE'], [obj[0].text, obj[1].text]))
    row_s = pd.Series(row)
    row_s.name = i
    dfSpeedMap = dfSpeedMap.append(row_s)

In [4]:
dfSpeedMap = dfSpeedMap.drop(dfSpeedMap.columns[[1,2, 3, 4, 5]], axis=1)
print dfSpeedMap[:20]

         LINK_ID
0     3006-30069
1   30069-888301
2    3010-888301
3      3363-3369
4     33671-3363
5     33681-3643
6     3368-88810
7    33691-33671
8     3369-33691
9      3369-3370
10    33701-3369
11    3370-34211
12    3393-88817
13     3395-3393
14     3402-8979
15     3417-3681
16     3419-3417
17   34211-33681
18   34212-33701
19   34212-34213


In [5]:
colsLatLong = ['Node', 'Latitude', 'Longitude'];
dataLatLong = pd.read_csv('/home/sohail/node-longlat.csv', names=colsLatLong, dtype={'Node': str, 'Latitude': str, 'Longitude':str})
dataLatLong 

newDF = dataLatLong.set_index(['Node'])

In [6]:
print newDF.xs('722')

Latitude     22.28599466
Longitude    114.1552444
Name: 722, dtype: object


In [7]:
dfSpeedMap['src_lat'] = np.nan
dfSpeedMap['src_long'] = np.nan
dfSpeedMap['des_lat'] = np.nan
dfSpeedMap['des_long'] = np.nan

dfSpeedMap.set_value(0,'src_lat',0)

,LINK_ID,src_lat,src_long,des_lat,des_long
0,3006-30069,0,NaN,NaN,NaN
1,30069-888301,NaN,NaN,NaN,NaN
2,3010-888301,NaN,NaN,NaN,NaN
3,3363-3369,NaN,NaN,NaN,NaN
4,33671-3363,NaN,NaN,NaN,NaN
5,33681-3643,NaN,NaN,NaN,NaN
6,3368-88810,NaN,NaN,NaN,NaN
7,33691-33671,NaN,NaN,NaN,NaN
8,3369-33691,NaN,NaN,NaN,NaN
9,3369-3370,NaN,NaN,NaN,NaN


In [8]:

for x in dfSpeedMap.iterrows():

    split_ID = str(x[1]["LINK_ID"]).split('-')
    
    dfSpeedMap.set_value(x[0],'src_lat',newDF.xs(split_ID[0])[0])
    dfSpeedMap.set_value(x[0],'src_long',newDF.xs(split_ID[0])[1])
    dfSpeedMap.set_value(x[0],'des_lat',newDF.xs(split_ID[1])[0])
    dfSpeedMap.set_value(x[0],'des_long',newDF.xs(split_ID[1])[1])
    
dfSpeedMap

,LINK_ID,src_lat,src_long,des_lat,des_long
0,3006-30069,22.306601,114.163728,22.309566,114.162857
1,30069-888301,22.309566,114.162857,22.309008,114.161770
2,3010-888301,22.310627,114.164769,22.309008,114.161770
3,3363-3369,22.325271,114.208454,22.325173,114.206014
4,33671-3363,22.324533,114.208630,22.325271,114.208454
5,33681-3643,22.319972,114.206819,22.317849,114.208611
6,3368-88810,22.319902,114.206735,22.320155,114.206450
7,33691-33671,22.324300,114.206159,22.324533,114.208630
8,3369-33691,22.325173,114.206014,22.324300,114.206159
9,3369-3370,22.325173,114.206014,22.325068,114.204617


In [9]:
dfSpeedMap.reset_index().to_json('/home/sohail/dfSpeedMap.json', orient='records')